In [24]:
from elasticsearch import helpers
from elasticsearch import Elasticsearch, TransportError
import argparse
import gzip
import json
import re
import traceback
from tqdm.auto import tqdm
import xmltodict
import urllib
import json
import sys

TREC_YEAR = 2021 # 2019, using V2 for both
TREC_DATAFILES_DIR = "/DATA/users/kenanfayoumi/EntityRanking/trec_files"

with open(f'{TREC_DATAFILES_DIR}/num_to_docid_mapping_TREC{TREC_YEAR}.json', 'r') as fp:
    num_to_docid_mapping=json.load(fp)
docid_to_num_mapping  = {v: k for k, v in num_to_docid_mapping.items()}




In [15]:
topics_jsons = []


with open("/DATA/projects/TRECNews/WAPO/WashingtonPost.v4/data/TREC_Washington_Post_collection.v4.jl",'rb') as f:
# with open("/DATA/projects/TRECNews/WAPO/WashingtonPost.v3/data/TREC_Washington_Post_collection.v3.jl",'rb') as f:
# with open("/DATA/projects/TRECNews/WAPO/WashingtonPost.v2/data/TREC_Washington_Post_collection.v2.jl",'rb') as f:
    for line in tqdm(f, total=700000):
        js = json.loads(line)
        if js['id'] in docid_to_num_mapping:
            topics_jsons.append(js)

In [17]:
trec_year = 2020

with open(f"data_files/{trec_year}_topics_json_contents.jsonl",'w') as f:
    for obj in topics_jsons:
        f.write(json.dumps(obj)+"\n")

# get article text contents

In [32]:
# we need to get the text content for all of these articles
needed_article_ids = []

In [41]:
# get all years qrels needed articles
articles_linking = dict()

qrels_file ="/DATA/projects/TRECNews/2018/bl/bqrels.exp-gains.txt"
with open(qrels_file,'r') as file:
  for line in file.readlines():
    splitted = line.split()
    links = []
    if(splitted[0] in articles_linking):
      links = articles_linking[splitted[0]]
    links.append([splitted[2],splitted[3]])
    links = sorted(links,key= lambda x: int(x[1]),reverse=True)
    articles_linking[splitted[0]] = links
    
qrels_file = "/DATA/projects/TRECNews/2019/bl/newsir19-qrels-background.txt"
with open(qrels_file,'r') as file:
  for line in file.readlines():
    splitted = line.split()
    links = []
    if(splitted[0] in articles_linking):
      links = articles_linking[splitted[0]]
    links.append([splitted[2],splitted[3]])
    links = sorted(links,key= lambda x: int(x[1]),reverse=True)
    articles_linking[splitted[0]] = links
    
qrels_file = "/DATA/projects/TRECNews/2020/bl/qrels.background"
with open(qrels_file,'r') as file:
  for line in file.readlines():
    splitted = line.split()
    links = []
    if(splitted[0] in articles_linking):
      links = articles_linking[splitted[0]]
    links.append([splitted[2],splitted[3]])
    links = sorted(links,key= lambda x: int(x[1]),reverse=True)
    articles_linking[splitted[0]] = links
    
qrels_file = "/DATA/projects/TRECNews/2021/bl/qrels.background"
with open(qrels_file,'r') as file:
  for line in file.readlines():
    splitted = line.split()
    links = []
    if(splitted[0] in articles_linking):
      links = articles_linking[splitted[0]]
    links.append([splitted[2],splitted[3]])
    links = sorted(links,key= lambda x: int(x[1]),reverse=True)
    articles_linking[splitted[0]] = links
    

with open(f"data_files/articles_linking.json",'w') as f:
    json.dump(articles_linking,f)
    
    
needed_article_ids += [pair[0] for list_of_pairs in articles_linking.values() for pair in list_of_pairs]

In [50]:
# test topics ids
TREC_DATAFILES_DIR = "/DATA/users/kenanfayoumi/EntityRanking/trec_files"

TREC_YEAR = 2018 # 2019, using V2 for both
with open(f'{TREC_DATAFILES_DIR}/num_to_docid_mapping_TREC{TREC_YEAR}.json', 'r') as fp:
    num_to_docid_mapping=json.load(fp)
needed_article_ids += list(num_to_docid_mapping.values())

TREC_YEAR = 2019 # 2019, using V2 for both
with open(f'{TREC_DATAFILES_DIR}/num_to_docid_mapping_TREC{TREC_YEAR}.json', 'r') as fp:
    num_to_docid_mapping=json.load(fp)
needed_article_ids += list(num_to_docid_mapping.values())

TREC_YEAR = 2020 # 2019, using V2 for both
with open(f'{TREC_DATAFILES_DIR}/num_to_docid_mapping_TREC{TREC_YEAR}.json', 'r') as fp:
    num_to_docid_mapping=json.load(fp)
needed_article_ids += list(num_to_docid_mapping.values())

TREC_YEAR = 2021 # 2019, using V2 for both
with open(f'{TREC_DATAFILES_DIR}/num_to_docid_mapping_TREC{TREC_YEAR}.json', 'r') as fp:
    num_to_docid_mapping=json.load(fp)
needed_article_ids += list(num_to_docid_mapping.values())

In [53]:
# also get all baseline candidates
with open('res_files/wapoV2_BASELINE_2018','r') as file:
  for line in file.readlines():
    splitted = line.split()
    needed_article_ids.append(splitted[2])

with open('res_files/wapoV2_BASELINE_2019','r') as file:
  for line in file.readlines():
    splitted = line.split()
    needed_article_ids.append(splitted[2])
    
    
with open('res_files/wapoV4_BASELINE_2020','r') as file:
  for line in file.readlines():
    splitted = line.split()
    needed_article_ids.append(splitted[2])
    
with open('res_files/wapoV4_BASELINE_2021','r') as file:
  for line in file.readlines():
    splitted = line.split()
    needed_article_ids.append(splitted[2])

In [57]:
needed_article_ids = list(set(needed_article_ids))

In [59]:
def get_all_content_by_type(jsarr, t, field='content'):
    strings = [c[field] for c in jsarr if c is not None and c['type'] == t and field in c and c[field] is not None]
    if strings:
        return ' '.join(strings)
    else:
        return None

In [61]:
article_texts = {}

with open("/DATA/projects/TRECNews/WAPO/WashingtonPost.v4/data/TREC_Washington_Post_collection.v4.jl",'rb') as f:
    for line in tqdm(f, total=700000):
        js = json.loads(line)
        if js['id'] in needed_article_ids:
            text = get_all_content_by_type(js['contents'], 'sanitized_html')
            if text:
                text = re.sub('<.*?>', ' ', text)
                article_texts[js['id']] = text
            
with open("/DATA/projects/TRECNews/WAPO/WashingtonPost.v3/data/TREC_Washington_Post_collection.v3.jl",'rb') as f:
# with open("/DATA/projects/TRECNews/WAPO/WashingtonPost.v2/data/TREC_Washington_Post_collection.v2.jl",'rb') as f:
    for line in tqdm(f, total=700000):
        js = json.loads(line)
        if js['id'] in needed_article_ids:
            text = get_all_content_by_type(js['contents'], 'sanitized_html')
            if text:
                text = re.sub('<.*?>', ' ', text)
                article_texts[js['id']] = text
            
            
with open("/DATA/projects/TRECNews/WAPO/WashingtonPost.v2/data/TREC_Washington_Post_collection.v2.jl",'rb') as f:
    for line in tqdm(f, total=700000):
        js = json.loads(line)
        if js['id'] in needed_article_ids:
            text = get_all_content_by_type(js['contents'], 'sanitized_html')
            if text:
                text = re.sub('<.*?>', ' ', text)
                article_texts[js['id']] = text
            

In [67]:
with open(f"data_files/article_texts.json",'w') as f:
    json.dump(article_texts,f)
    